In [1]:
import sys
import os
# hardcodeo de paths
sys.path.append(os.path.dirname(os.path.abspath('')))
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

import pickle
import pandas as pd
import numpy as np
import gower

# for sickit gower:
# para sickit gower necesito hacer one hot encoding y además llenar todos los NA porque no los soporta

#from sklearn.metrics import pairwise_distances
#import scipy.sparse as sp
#import warnings
#from numpy.core.numeric import ComplexWarning
#from scipy.spatial import distance
#from scipy.sparse import csr_matrix
#from scipy.sparse import issparse

from sklearn.manifold import MDS

from matplotlib import pyplot as plt
import seaborn as sns         
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


In [2]:
llamados= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_group_I.xlsx'))


In [3]:
# droppear las variables que están agrupadas en otras, fecha y hora separadas y provincia id

llamados.drop(['vs_explotacion_sexual','vs_explotacion_sexual_comercial','vs_explotacion_sexual_viajes_turismo',
              'vs_sospecha_trata_personas_fines_sexuales', 'vs_violacion_via_vaginal', 'vs_violacion_via_anal', 
               'vs_violacion_via_oral', 'ofv_uso_arma_blanca','ofv_uso_arma_fuego', 'ofv_intento_ahorcar', 
               'ofv_intento_quemar', 'ofv_intento_ahogar','ofv_intento_matar', 'llamado_fecha', 'llamado_hora',
              'llamado_provincia_id'], axis=1, inplace=True)



In [4]:
# corregir nombres de columnas (TO DO: pasar al pipeline de agrupación luego)

llamados.rename(columns={"ofv_uso_arma": "ofv_uso_arma_group", "ofv_intento_violencia_fisica": "ofv_intento_violencia_fisica_group"}, inplace=True)
#llamados.columns

In [3]:
llamados['victima_genero'] = llamados['victima_genero'].fillna('NS/NC')

In [4]:
llamados.columns

Index(['llamado_fecha_hora', 'llamado_fecha', 'llamado_hora',
       'llamado_provincia', 'llamado_provincia_id', 'llamante_edad',
       'llamante_genero', 'llamante_vinculo', 'caso_judicializado',
       'hecho_lugar', 'victima_a_resguardo', 'victima_edad', 'victima_genero',
       'victima_nacionalidad', 'victima_vinculo_agresor',
       'victima_discapacidad', 'victima_convive_agresor',
       'vs_violacion_via_vaginal', 'vs_violacion_via_anal',
       'vs_violacion_via_oral', 'vs_tentativa_violacion',
       'vs_tocamiento_sexual', 'vs_intento_tocamiento',
       'vs_Intento_violación_tercera_persona', 'vs_grooming',
       'vs_exhibicionismo', 'vs_amenazas_verbales_contenido_sexual',
       'vs_explotacion_sexual', 'vs_explotacion_sexual_comercial',
       'vs_explotacion_sexual_viajes_turismo',
       'vs_sospecha_trata_personas_fines_sexuales',
       'vs_existencia_facilitador_corrupcion_nnya',
       'vs_obligacion_sacarse_fotos_pornograficas',
       'vs_eyaculacion_partes_c

In [8]:
cols = [col for col in llamados.columns if col not in ['llamante_edad', 'victima_edad', 'llamado_provincia_id', 'llamado_fecha', 'llamado_hora']]
llamados_2 = llamados[cols]

NameError: name 'llamados' is not defined

In [10]:
#llamados_2.columns

In [6]:
del llamados

In [7]:
gower_data = gower.gower_matrix(llamados_2)
gower_data

array([[0.        , 0.25454545, 0.23636363, ..., 0.25454545, 0.21818182,
        0.21818182],
       [0.25454545, 0.        , 0.21818182, ..., 0.18181819, 0.18181819,
        0.2       ],
       [0.23636363, 0.21818182, 0.        , ..., 0.27272728, 0.23636363,
        0.25454545],
       ...,
       [0.25454545, 0.18181819, 0.27272728, ..., 0.        , 0.12727273,
        0.25454545],
       [0.21818182, 0.18181819, 0.23636363, ..., 0.12727273, 0.        ,
        0.18181819],
       [0.21818182, 0.2       , 0.25454545, ..., 0.25454545, 0.18181819,
        0.        ]], dtype=float32)

In [9]:
def mapData(dist_matrix, X, y, metric, title):
    mds = MDS(metric=metric, dissimilarity='precomputed', random_state=0) # this is the second step to NMDS or MDS
    # Get the embeddings
    pts = mds.fit_transform(dist_matrix)
    # Plot the embedding, colored according to the class of the points
    fig = plt.figure(2, (15,6))
    ax = fig.add_subplot(1,2,1) 
    
    # pts[:, 0] pts in column 1 (first dimension),y=pts[:, 1] pts in column 2 (second dimension) 

    ax = sns.scatterplot(x=pts[:, 0], y=pts[:, 1], hue=y, palette=['blue', 'red'], hue_order=['NO', 'YSI'])


    plt.title(title)    
    plt.show()

In [10]:
# mapping SI/NO in "victima_convive_agresor" labels to 'y_convive'
y_convive = []
for value in llamados_2['victima_convive_agresor']:
    if value == 'SI':
        y_convive.append('SI')
    elif value == 'NO':
        y_convive.append('NO')

In [ ]:
mapData(gower_data, llamados_2, y_convive, False, 
        'Non-metric MDS with Gower')


### Reemplazar nan en victima_genero por NS/NC

### Hacer el ordenamiento sin las columnas de edad jeje